In [12]:

import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.plot import show
import numpy as np
import os
import pandas as pd
from shapely.geometry import Point, Polygon
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats import gaussian_kde

### Purpose and Description of this Notebook:
This notebook aims to clean and preprocess the Ethiopian Health Facilities dataset Ethiopian Health Facilities.shp.
The cleaning process involves handling missing values, standardizing text fields, and removing duplicates to ensure data consistency and quality.
Additionally, this notebook merges the cleaned Ethiopian Health Facilities dataset with the AddisDatabase AddisDatabase.shp.
The merged dataset will provide a comprehensive view of health facilities in Ethiopia, particularly in Addis Ababa, enabling more accurate analysis and decision-making.


# Data Cleaning
This section handles the cleaning and preprocessing of the Ethiopian health facilities dataset including handling missing values, standardizing text fields, and removing duplicates


In [13]:
# Load the data
ethiopia_health = gpd.read_file('G:\\My Drive\\Data\\Ethiopian_health_Facilities\\Ethiopian Health Facilities.shp')

# Filter the data to include only Addis Ababa
addis_ababa_health = ethiopia_health[ethiopia_health['admin1Name'] == 'Addis Ababa']

# Clean the data
# 1. Remove rows with missing critical information
addis_ababa_health = addis_ababa_health.dropna(subset=['Type', 'admin1Name', 'Ownership'])

# 2. Standardize text columns
# List of text columns to standardize
text_columns = ['Type', 'admin1Name', 'admin2Name', 'Name', 'admin3Name', 'City', 'Kebele', 'Ownership']

# Standardize each text column
for col in text_columns:
    # Apply basic string cleaning (strip whitespace) to all columns
    addis_ababa_health[col] = addis_ababa_health[col].str.strip()
    # addis_ababa_health[col] = addis_ababa_health[col].str.title()
    
    
    
# 3. Create health facility counts by administrative levels
# Check for duplicates and drop them, keeping only unique entries
duplicate_check = addis_ababa_health.duplicated(keep=False)
if duplicate_check.any():
    print(f"Found and removing {duplicate_check.sum()} duplicate entries")
    # Drop duplicates and keep first occurrence
    addis_ababa_health = addis_ababa_health.drop_duplicates(keep='first')
    print(f"Remaining unique entries: {len(addis_ababa_health)}")

# Create health facility counts
health_counts = addis_ababa_health.groupby(['Type', 'admin1Name', 'admin2Name', 'admin3Name'])\
    .size()\
    .reset_index(name='health_institution_count')\
    .sort_values('health_institution_count', ascending=False)

# 4. Create ownership distribution
ownership_counts = addis_ababa_health.groupby(['admin1Name', 'Ownership'])\
    .size()\
    .reset_index(name='ownership_count')\
    .sort_values('ownership_count', ascending=False)
print(health_counts)
print(ownership_counts)



                                               Type   admin1Name admin2Name  \
68                                         Pharmacy  Addis Ababa  Region 14   
69                                         Pharmacy  Addis Ababa  Region 14   
71                                         Pharmacy  Addis Ababa  Region 14   
65                                         Pharmacy  Addis Ababa  Region 14   
63                                         Pharmacy  Addis Ababa  Region 14   
..                                              ...          ...        ...   
57  Medical Office Practice,Medical Office Practice  Addis Ababa  Region 14   
46                                     Laboratories  Addis Ababa  Region 14   
50                                     Laboratories  Addis Ababa  Region 14   
52                                     Laboratories  Addis Ababa  Region 14   
73                                 Specialty Center  Addis Ababa  Region 14   

          admin3Name  health_institution_count  
68

In [14]:
# Load Addis Ababa shapefile
addis_db = gpd.read_file('G:\\My Drive\\AddisDatabase May 5 2025\\AddisDatabase May 5 2025\\AddisDatabase.shp')

# Drop rows with missing Woreda values
addis_db = addis_db.dropna(subset=['Woreda'])

# Check for duplicates and drop them
duplicate_check = addis_db.duplicated(keep=False)
if duplicate_check.any():
    print(f"Found and removing {duplicate_check.sum()} duplicate entries")
    # Drop duplicates and keep first occurrence
    addis_db = addis_db.drop_duplicates(keep='first')
    print(f"Remaining unique entries: {len(addis_db)}")

# Convert CRS to EPSG:4326 (WGS84)
addis_db = addis_db.to_crs(epsg=4326)




In [15]:

# Double check the CRS
print(addis_db.crs)
print(addis_ababa_health.crs)

addis_health_merged = gpd.sjoin(addis_db, addis_ababa_health, how='left', predicate='intersects')
addis_health_merged

EPSG:4326
EPSG:4326


,Woreda,Sub_City,City_left,popden,Area,geometry,index_right,Id,Name,Latitude,...,Type,admin3Name,admin3Pcod,admin2Name,admin2Pcod,admin1Name,admin1Pcod,City_right,Kebele,Status
0,W01,Yeka,Addis,159.881253,394.192424,"POLYGON ((38.78806 9.08043, 38.78802 9.08041, ...",15525.0,1029902.0,Fraol pharmacy,9.054760,...,Pharmacy,Yeka,ET140107,Region 14,ET1401,Addis Ababa,ET14,Addis Ababa,None,Approved
0,W01,Yeka,Addis,159.881253,394.192424,"POLYGON ((38.78806 9.08043, 38.78802 9.08041, ...",22939.0,1029082.0,Kenean Medium Clinic,9.062969,...,Clinic,Yeka,ET140107,Region 14,ET1401,Addis Ababa,ET14,Addis Ababa,None,Approved
0,W01,Yeka,Addis,159.881253,394.192424,"POLYGON ((38.78806 9.08043, 38.78802 9.08041, ...",7711.0,1044399.0,Biniyam Medium Clinic,9.064228,...,Clinic,Yeka,ET140107,Region 14,ET1401,Addis Ababa,ET14,Addis Ababa,None,Approved
0,W01,Yeka,Addis,159.881253,394.192424,"POLYGON ((38.78806 9.08043, 38.78802 9.08041, ...",39569.0,1029099.0,Yewodekutn Ansu Medium Clinic,9.067579,...,Clinic,Yeka,ET140107,Region 14,ET1401,Addis Ababa,ET14,Addis Ababa,None,Approved
0,W01,Yeka,Addis,159.881253,394.192424,"POLYGON ((38.78806 9.08043, 38.78802 9.08041, ...",27656.0,1029104.0,Meri Ferensaye Medium Clinic,9.056789,...,Clinic,Yeka,ET140107,Region 14,ET1401,Addis Ababa,ET14,Addis Ababa,None,Approved
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,10,Addis ketema,Addis,52.491053,125.375800,"POLYGON ((38.71495 9.05432, 38.71504 9.05432, ...",10342.0,1029958.0,Daniya Pharmacy,9.041535,...,Pharmacy,Addis Ketema,ET140108,Region 14,ET1401,Addis Ababa,ET14,Addis Ababa,12,Approved
115,10,Addis ketema,Addis,52.491053,125.375800,"POLYGON ((38.71495 9.05432, 38.71504 9.05432, ...",39684.0,1057890.0,Yinatu pharmacy,9.041891,...,Pharmacy,Addis Ketema,ET140108,Region 14,ET1401,Addis Ababa,ET14,Addis Ababa,None,Approved
115,10,Addis ketema,Addis,52.491053,125.375800,"POLYGON ((38.71495 9.05432, 38.71504 9.05432, ...",28059.0,1030275.0,Miback Pharmacy,9.042422,...,Pharmacy,Addis Ketema,ET140108,Region 14,ET1401,Addis Ababa,ET14,Addis Ababa,10,Approved
115,10,Addis ketema,Addis,52.491053,125.375800,"POLYGON ((38.71495 9.05432, 38.71504 9.05432, ...",14306.0,1030209.0,Ethio Marble Primarly Clinic,9.054726,...,Clinic,Gulele,ET140110,Region 14,ET1401,Addis Ababa,ET14,Addis Ababa,10,Approved


In [16]:
addis_health_merged = addis_health_merged[['Woreda', 'Sub_City', 'popden', 'Area', 'Name', 'Ownership', 'Type', 'Status', 'geometry']]

# Data Engineering 
Here we create additional columns of number of health institutions by type, ownership, and status



In [17]:
addis_health_merged

,Woreda,Sub_City,popden,Area,Name,Ownership,Type,Status,geometry
0,W01,Yeka,159.881253,394.192424,Fraol pharmacy,Private for profit,Pharmacy,Approved,"POLYGON ((38.78806 9.08043, 38.78802 9.08041, ..."
0,W01,Yeka,159.881253,394.192424,Kenean Medium Clinic,Private for profit,Clinic,Approved,"POLYGON ((38.78806 9.08043, 38.78802 9.08041, ..."
0,W01,Yeka,159.881253,394.192424,Biniyam Medium Clinic,Private for profit,Clinic,Approved,"POLYGON ((38.78806 9.08043, 38.78802 9.08041, ..."
0,W01,Yeka,159.881253,394.192424,Yewodekutn Ansu Medium Clinic,Other Government,Clinic,Approved,"POLYGON ((38.78806 9.08043, 38.78802 9.08041, ..."
0,W01,Yeka,159.881253,394.192424,Meri Ferensaye Medium Clinic,Private for profit,Clinic,Approved,"POLYGON ((38.78806 9.08043, 38.78802 9.08041, ..."
...,...,...,...,...,...,...,...,...,...
115,10,Addis ketema,52.491053,125.375800,Daniya Pharmacy,Private for profit,Pharmacy,Approved,"POLYGON ((38.71495 9.05432, 38.71504 9.05432, ..."
115,10,Addis ketema,52.491053,125.375800,Yinatu pharmacy,Private for profit,Pharmacy,Approved,"POLYGON ((38.71495 9.05432, 38.71504 9.05432, ..."
115,10,Addis ketema,52.491053,125.375800,Miback Pharmacy,Private for profit,Pharmacy,Approved,"POLYGON ((38.71495 9.05432, 38.71504 9.05432, ..."
115,10,Addis ketema,52.491053,125.375800,Ethio Marble Primarly Clinic,Private Not for profit,Clinic,Approved,"POLYGON ((38.71495 9.05432, 38.71504 9.05432, ..."


In [18]:
# Step 1: Count total number of health institutions by Sub_City and Woreda
count_total = addis_health_merged.groupby(['Woreda', 'Sub_City', 'popden', 'Area', 'geometry']).size().reset_index(name='number_of_health_inst')

# Step 2: Pivot Ownership, Type, and Status into counts
pivot_ownership = pd.pivot_table(addis_health_merged, index=['Woreda', 'Sub_City', 'popden', 'Area', 'geometry'], columns='Ownership', aggfunc='size', fill_value=0)
pivot_type = pd.pivot_table(addis_health_merged, index=['Woreda', 'Sub_City', 'popden', 'Area', 'geometry'], columns='Type', aggfunc='size', fill_value=0)
pivot_status = pd.pivot_table(addis_health_merged, index=['Woreda', 'Sub_City', 'popden', 'Area', 'geometry'], columns='Status', aggfunc='size', fill_value=0)

# Step 3: Combine all into a condensed dataset
condensed_df = count_total \
    .set_index(['Woreda', 'Sub_City', 'popden', 'Area', 'geometry']) \
    .join([pivot_ownership, pivot_type, pivot_status]) \
    .reset_index()
condensed_df

,Woreda,Sub_City,popden,Area,geometry,number_of_health_inst,Other Government,Private Not for profit,Private for profit,Public/Government,...,Laboratories,Medical Office Practice,"Medical Office Practice,Medical Office Practice",Other,"Other,Home-Based Care Service",Pharmacy,Specialty Center,Specialty Clinic,Approved,Denied
0,01,Addis ketema,126.697300,76.678391,"POLYGON ((38.74371 9.03294, 38.74369 9.03294, ...",3,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0
1,01,Bole,126.697300,828.364928,"POLYGON ((38.81233 8.98612, 38.8124 8.98597, 3...",31,1.0,0.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18.0,1.0,5.0,31.0,0.0
2,01,Cherkos,126.697300,139.465876,"POLYGON ((38.77472 9.01096, 38.77497 9.0102, 3...",15,0.0,1.0,14.0,0.0,...,2.0,0.0,0.0,0.0,0.0,3.0,0.0,7.0,15.0,0.0
3,02,Addis ketema,186.715539,37.847492,"POLYGON ((38.73355 9.02312, 38.73354 9.02313, ...",5,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0
4,02,Bole,186.715539,117.217250,"POLYGON ((38.79152 8.98756, 38.79132 8.98737, ...",22,1.0,0.0,19.0,2.0,...,0.0,0.0,0.0,0.0,0.0,4.0,3.0,10.0,22.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,w12,Kolfe Keranyo,132.385765,71.341558,"POLYGON ((38.71191 9.04426, 38.71187 9.04417, ...",11,0.0,0.0,11.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,11.0,0.0
112,w12,Nifas Silk Lafto,132.385765,200.429296,"POLYGON ((38.74423 8.96062, 38.74451 8.96061, ...",27,0.0,0.0,27.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20.0,1.0,2.0,27.0,0.0
113,w13,Kolfe Keranyo,148.424780,301.079733,"POLYGON ((38.71258 9.05797, 38.71272 9.05793, ...",18,0.0,0.0,16.0,2.0,...,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,18.0,0.0
114,w14,Kolfe Keranyo,127.162128,221.762326,"POLYGON ((38.69559 9.06611, 38.6955 9.06598, 3...",11,0.0,0.0,11.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,2.0,11.0,0.0


In [19]:
condensed_df.columns

Index(['Woreda', 'Sub_City', 'popden', 'Area', 'geometry',
       'number_of_health_inst', 'Other Government', 'Private Not for profit',
       'Private for profit', 'Public/Government', 'Clinic', 'Health Center',
       'Health Post', 'Home-Based Care Service', 'Hospital', 'ImagingCenters',
       'Laboratories', 'Medical Office Practice',
       'Medical Office Practice,Medical Office Practice', 'Other',
       'Other,Home-Based Care Service', 'Pharmacy', 'Specialty Center',
       'Specialty Clinic', 'Approved', 'Denied'],
      dtype='object')

# Data Data Visualization
Task for Tarik- Here we map the popdn heat map by placing the rank of each of the number of institutions columns including popden at subcity and woreda level. So you basically produce 2 maps one at sub_city and another at woreda level that summarize the heat map of pop_den and the number of institutions in a form of ranking.
